In [1]:
# Goal: Generate content for music inspired cocktails
template = {
    "title": "", # get list of song names. find rhymes for each word. replace closest word to alcohol. 
    "description": "", # tracery grammar 
    "brewed_date": "", # date of song
    "style": "", # music genre
    "alcohol_content": "", # random num?
    "notes": "", # list of misc liqeurs
    "pairing": "", # find list of song names. find rhymes for each word, replace closest word to X food
    "serving_suggestion": "" # tracery grammar using words from lyrics (closest words to temperature etc.)
}

In [115]:
# Download data, shell script
# song lyrics
! wget -O data/lyrics.csv \
https://raw.githubusercontent.com/walkerkq/musiclyrics/master/billboard_lyrics_1964-2015.csv
# Load wine descriptions
! wget https://raw.githubusercontent.com/dariusk/corpora/master/data/foods/wine_descriptions.json --directory-prefix=data
# # Load beer categories
! wget https://raw.githubusercontent.com/dariusk/corpora/master/data/foods/beer_categories.json --directory-prefix=data
# Load beer styles
! wget https://raw.githubusercontent.com/dariusk/corpora/master/data/foods/beer_styles.json --directory-prefix=data
# Load condiments
! wget https://raw.githubusercontent.com/dariusk/corpora/master/data/foods/condiments.json --directory-prefix=data
# Load breads_and_pastries
! wget https://raw.githubusercontent.com/dariusk/corpora/master/data/foods/breads_and_pastries.json --directory-prefix=data
# Load menuItems
! wget https://raw.githubusercontent.com/dariusk/corpora/master/data/foods/menuItems.json --directory-prefix=data
# Load cocktail images
! wget --adjust-extension \
     --random-wait \
     --span-hosts \
     --convert-links \
     --backup-converted \
     --no-directories \
     --timestamping \
     --page-requisites \
     --directory-prefix=data/images \
     --execute robots=off \
     -A "*.jpg" \
     'http://www.thecocktaildb.com/index.php'

--2017-09-25 13:02:39--  http://www.thecocktaildb.com/index.php
Resolving www.thecocktaildb.com... 104.27.190.164, 104.27.191.164, 2400:cb00:2048:1::681b:bea4, ...
Connecting to www.thecocktaildb.com|104.27.190.164|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/images/index.php.tmp.html’

index.php.tmp.html      [ <=>                ]   9.20K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2017-09-25 13:02:39 (170 MB/s) - ‘data/images/index.php.tmp.html’ saved [9424]

Removing data/images/index.php.tmp.html since it should be rejected.

--2017-09-25 13:02:39--  http://www.thecocktaildb.com/images/media/drink/vr6kle1504886114.jpg
Reusing existing connection to www.thecocktaildb.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 46409 (45K) [image/jpeg]
Saving to: ‘data/images/vr6kle1504886114.jpg’

vr6kle1504886114.jp 100%[===================>]  45.32K  69.8KB/s    in 0.6s   

In [617]:
# Create json of cocktail info
import requests
from bs4 import BeautifulSoup, Comment

data = {
    'cocktails': {},
    'source': 'http://www.cocktaildb.com'
}

def return_text(x):
    return x.get_text()

for i in range(4758):
    if (i % 10 == 0): # download every 10th entry to save time
        url = 'http://www.cocktaildb.com/recipe_detail?id={}'.format(i+1)
        raw_html = requests.get(url).text
        soup = BeautifulSoup(raw_html, 'html.parser')
        if (soup):
            cocktail = soup.find(id='wellTitle')
            if (cocktail):
                c_name = cocktail.find('h2').get_text()
                data['cocktails'][c_name] = {
                    'recipe': list(map(return_text, soup.find_all(class_='recipeMeasure'))),
                    'directions': list(map(return_text, soup.find_all(class_='recipeDirection')))
                }
                print(i)

import json
with open('data/cocktails.json', 'w') as outfile:
    json.dump(data, outfile)
    print('File written')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [1]:
#  Read lyrics names
import csv
lyrics_raw = csv.DictReader(open('data/lyrics.csv',  encoding='ISO-8859-1'))
lyrics = []
for row in lyrics_raw:
    # Keys: Rank, Song, Artist, Year, Lyrics, Source
    not_obscure = (int(row['Year']) >= 2010 and int(row['Rank']) < 50) or (int(row['Rank']) <= 5 and int(row['Year']) >= 1994)
    if (not_obscure) :
        lyrics.append({
            'rank': row['Rank'],
            'song': row['Song'],  
            'artist': row['Artist'],
            'year': row['Year'],
            'lyrics': row['Lyrics'],
            'source': row['Source']
        });


In [2]:
# NLP vector helpers
def vec(s):
    return nlp.vocab[s].vector

import numpy as np
from numpy import dot
from numpy.linalg import norm

# cosine similarity
def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0
    
def spacy_closest(token_list, vec_to_check, n=30):
    return sorted(token_list,
                  key=lambda x: cosine(vec_to_check, vec(x)),
                  reverse=True)[:n]


def word_count(token_list):
    count = {} 
    for token in token_list:
        text = str(token.text).strip()
        if (len(text) > 0):
            if (text not in count.keys()):
                count[text] = 0

            count[text] += 1
    return count

def addv(coord1, coord2):
    return [c1 + c2 for c1, c2 in zip(coord1, coord2)]

In [156]:
import random
import os
from __future__ import unicode_literals
import spacy
nlp = spacy.load('en')
import tracery
from tracery.modifiers import base_english
import json

# Misc Helpers ======================
def get_flavour_nouns(drink):
    drink_recipe_doc = nlp(' '.join(drink['recipe']))
    drink_recipe_tokens = list(set([w.text for w in drink_recipe_doc if w.is_alpha]))
    flavours = []
    # extract list of flavours from recipe
    for w in drink_recipe_doc:
        if (w.pos_ in ['NOUN', 'PROPN'] and w.text.lower() not in flavours and w.text.lower() not in ['sec', 'cl', 'gills', 'tsp', 'oz', 'splash', 'dashes', 'teaspoon', 'dash']):
            if(w.text.lower() == 'vermouth'):
                flavours.append('{} vermouth'.format('dry'))
            else:
                flavours.append(w.text)            
    return flavours

def get_vocab(s, d): # create a vocabulary from the lyrics based loosely on the recipe of the drink
    lyric_doc = nlp(s['lyrics'])
    lyric_tokens = list(set([w.text for w in lyric_doc if w.is_alpha]))
    spacy_closest(lyric_tokens, vec('flavour'))
    flavours = get_flavour_nouns(d);
    # for each flavour identified, find closest words in lyrics and collect by part of speech 
    flav_words = {}
    for flav in flavours:
        neighbours = spacy_closest(lyric_tokens, vec(flav))
        if (len(neighbours) > 0):
            neighbour_doc = nlp(' '.join(neighbours))
            for i in neighbour_doc:
                noun_blacklist = ['what', 'bought', 'm', 'tres', 'gon']
                adj_blacklist = ['nt', 'whatever', 'youyou', 'half', 'tell', 'ya', 'm', 'its', 'my', 'betweenchorus', 'i', 'you', 'me', 'homeand', 'most', 'least', 'second', 'tryna', 'their', 'her', 'want', 'moment', 'enough', 'much', 'sharpen', 'that', 'better', 'your', 'what', 'his', 'yours', 'wanna', 'last', 'my', 'give', 'got']
                if (i.pos_ == 'ADJ' and (i.text.lower().strip() not in adj_blacklist)):
                    if (i.pos_ not in flav_words.keys()):
                        flav_words[i.pos_] = []  
                    flav_words[i.pos_].append(i.text)
                elif (i.pos_ == 'NOUN' and (i.text.lower().strip() not in noun_blacklist)):
                    if (i.pos_ not in flav_words.keys()):
                        flav_words[i.pos_] = []  
                    flav_words[i.pos_].append(i.text)                    
                elif (i.pos_ != 'ADJ' and i.pos_ != 'NOUN'):
                    if (i.pos_ not in flav_words.keys()):
                        flav_words[i.pos_] = []  
                    flav_words[i.pos_].append(i.text)
    return flav_words

def get_foods():
    data = {}
    data['condiment'] = json.loads(open("./data/condiments.json").read())['condiments']
    data['bread'] = json.loads(open("./data/breads_and_pastries.json").read())['breads']
    data['pastry'] = json.loads(open("./data/breads_and_pastries.json").read())['pastries']
    data['menu_item'] = json.loads(open("./data/menuItems.json").read())['menuItems']
    data['wine_descriptions'] = json.loads(open("./data/wine_descriptions.json").read())['wine_descriptions']
    data['beer_categories'] = json.loads(open("./data/beer_categories.json").read())['beer_categories']
    data['beer_styles'] = json.loads(open("./data/beer_styles.json").read())['beer_styles']
    return data

def humanise_name(n):
    return n.lower().replace('featuring', '&').replace('and', '&').replace('"', '').strip().title().replace('Jayz', 'Jay-Z').replace('Keha', 'Ke$ha')

# Generate content ======================
def get_drink_title(s, d):
    case = random.randint(0, 4)
    if (len(s['song'].split()) > 1 and case == 0):  # replace alcohol with artist name
        nlp = spacy.load('en')
        song_title_doc = nlp(s['song'])
        drink_doc = nlp(d['name'])
        case2 = random.randint(0, 1)
        name = []
        replaced = False
        for w in song_title_doc:
            if ((w.pos_ == 'NOUN' or w.pos_ == 'ADJ' or w.pos_ == 'PRON') and replaced == False):
                drink_tokens = list(set([z.text for z in drink_doc if z.is_alpha]))
                neighbours = spacy_closest(drink_tokens, vec('liquour'))
                if (len(neighbours) >= 1):
                    alcohol_name = neighbours[0]
                    name.append(alcohol_name)
                    replaced = True
            else:
                name.append(w.text)
        apostrophe = '\'' if (s['artist'][-1].lower() == 's') else '\'s'
        return '{}{} {}'.format(humanise_name(s['artist']), apostrophe, humanise_name(' '.join(name)))
    elif (case == 1): # artist's song name
        apostrophe = '\'' if (s['artist'][-1].lower() == 's') else '\'s'
        return '{}{} {}'.format(humanise_name(s['artist']), apostrophe, humanise_name(s['song']))      
    else: # artist's #DRINK#
        apostrophe = '\'' if (s['artist'][-1].lower() == 's') else '\'s'
        return '{}{} {}'.format(humanise_name(s['artist']), apostrophe, d['name'])

def get_drink_description(s, d):
    description_rules = get_foods()
    description_rules['artist'] = humanise_name(s['artist'])
    description_rules['drink1'] = d['name']
    description_rules['drink2'] = random.choice(list(drink_data['cocktails'].items()))[0]
    description_rules['first_sentence'] = ['#ADJ.a.capitalize# mix between the #drink1# and the #drink2#', '#ADJ.a.capitalize# variant of the #drink1#']
    vocab = get_vocab(s, d)
    if ('ADJ' in vocab.keys()):
        description_rules['ADJ'] = vocab['ADJ']
    else:
        description_rules['ADJ'] = ['aged', 'worn', 'weathered']
        
    if ('NOUN' in vocab.keys()):
        description_rules['NOUN'] = vocab['NOUN']
    else:
        description_rules['NOUN'] = ['flow', 'music', 'crib']
        
    apostrophe = '\'' if (s['artist'][-1].lower() == 's') else '\'s'
        
    description_rules['base'] = "#first_sentence#. As #ADJ# as #artist#{} #NOUN#.".format(apostrophe)  
    description = tracery.Grammar(description_rules)
    description.add_modifiers(base_english)
    return description.flatten('#base#')

def get_pairing(s, d):
    vocab = get_vocab(s, d)
    pairing_rules = get_foods()
    pairing_rules['artist'] = s['artist']
    if ('ADJ' in vocab.keys()):
        pairing_rules['ADJ'] = vocab['ADJ']
    else:
        pairing_rules['ADJ'] = ['aged', 'worn', 'weathered']
        
    plural_artist = 'has'
    if ('&' in humanise_name(s['artist'])):
        plural_artist = 'have'
    pairing_rules['base'] = "{} it with #ADJ.a# #menu_item# and #pastry#.".format('{} {}'.format(humanise_name(s['artist']), plural_artist))
    
    pairing = tracery.Grammar(pairing_rules)
    pairing.add_modifiers(base_english)
    return pairing.flatten('#base#')
    
def get_drink_style(s, d):
    description_rules = get_vocab(s, d)
    if ('ADJ' not in description_rules.keys()):
        description_rules['ADJ'] = ['passable', 'tender', 'loud']
    
    description_rules['second_adj'] = ['', ' and #ADJ#', ', #ADJ# and #ADJ#']
    description_rules['base'] = ['#ADJ.capitalize##second_adj#.']
    description = tracery.Grammar(description_rules)
    description.add_modifiers(base_english)
    return description.flatten('#base#')

def get_drink_notes(s, d):
    vocab = get_vocab(s, d)
    if ('NOUN' not in vocab.keys()):
        vocab['NOUN'] = ['hair', 'music', 'party']
        
    note_noun = random.choice(vocab['NOUN']).title()
    
    flavours = get_flavour_nouns(d)
    notes_text = []
    if (len(flavours) > 0):
        for i in range(len(flavours)):
            if (i < 2):
                notes_text.append(flavours[i].title())
    return (str(', '.join(notes_text)) + ' and a {} of {}.'.format(random.choice(['hint', 'dash']), note_noun.lower()))
    
def get_drink_serving(s, d):
    vocab = get_vocab(s, d)
    if ('ADJ' not in vocab.keys()):
        vocab['ADJ'] = ['passable', 'tender', 'loud']
        
    if(len(d['directions']) > 0):
        serving = random.choice(d['directions'])
        return '{}. {}.'.format(serving, 'Drink to the {} sounds of {}'.format(random.choice(vocab['ADJ']), s['song'].title()))
    else:
        return random.choice(['Enjoy cold.', 'On the rocks.'])

def get_image(i = 'z'):
    images = os.listdir('./data/images')
    if (i == 'z'):
        return random.choice(images)
    else:
        return images[i % len(images)]

def get_drink(s, d, i):
    content = {
        'title': get_drink_title(s, d),
        'description': get_drink_description(s, d),
        'brewed_date': s['year'],
        'style': get_drink_style(s, d),
        'alcohol_content': "{}%".format(round(random.random() + float(s['rank']), 1)),
        'notes': get_drink_notes(s, d),
        'pairing': get_pairing(s, d),
        'serving_suggestion': get_drink_serving(s, d),
        'image': get_image(i)
    }
    return content

for i in range(6):
    # Selecting random data ======================
    drink_data = {};
    with open('data/cocktails.json') as d:
        data = json.load(d)
        d.close()
        drink_data = data

    song = random.choice(lyrics)
    song['song'].replace('suit  tie', 'suit and tie')

    d = random.choice(list(drink_data['cocktails'].items()))
    drink = {
        'name': d[0],
        'directions': d[1]['directions'],
        'recipe': d[1]['recipe']   
    }
    # Generate content
    print(get_drink(song, drink, i), '\n')

{'title': "Dj Snake & Lil Jon's Turn Down For Hot", 'description': "A loud variant of the Hot Springs. As whatfire as Dj Snake & Lil Jon's shotsturn.", 'brewed_date': '2014', 'style': 'Loud, loud and whatfire.', 'alcohol_content': '15.8%', 'notes': 'Wine, Pineapple and a hint of shotsturn.', 'pairing': 'Dj Snake & Lil Jon have it with a whatfire French Coffee and éclair.', 'serving_suggestion': 'Shake in iced cocktail shaker & strain. Drink to the round sounds of Turn Down For What.', 'image': '1q7coh1504736227.jpg'} 

{'title': "Ke$ha's Blow", 'description': "A bored variant of the Ulanda. As bored as Ke$ha's togo.", 'brewed_date': '2011', 'style': 'Bored, bored and bored.', 'alcohol_content': '33.7%', 'notes': 'Gin, Cointreau and a dash of names.', 'pairing': 'Ke$ha has it with a bored Eggs, boiled and cronut.', 'serving_suggestion': 'Serve in a cocktail glass (4.5 oz). Drink to the bored sounds of Blow.', 'image': '2bcase1504889637.jpg'} 

{'title': "Zedd & Foxes' Clarity", 'descrip